In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_and_preprocess_data(data_path):
    all_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.csv')]
    df_list = []
    for file in all_files:
        df = pd.read_csv(file, encoding='latin1')
        df.columns = df.columns.str.strip()
        df_list.append(df)
    
    full_df = pd.concat(df_list, ignore_index=True)
    
    # 무한대 값을 NaN으로 바꾸고, NaN 값을 포함하는 행 제거
    full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    full_df.dropna(inplace=True)
    
    # 숫자형이 아닌 열과 레이블 열 제외
    numeric_cols = full_df.select_dtypes(include=np.number).columns.tolist()
    
    # 정상 데이터만 선택
    benign_df = full_df[full_df['Label'] == 'BENIGN']
    attack_df = full_df[full_df['Label'] != 'BENIGN']
    
    # 스케일링
    scaler = MinMaxScaler()
    benign_scaled = scaler.fit_transform(benign_df[numeric_cols])
    attack_scaled = scaler.transform(attack_df[numeric_cols])
    
    return benign_scaled, attack_scaled, benign_df, attack_df, numeric_cols, scaler

data_path = 'dataset/'
benign_data, attack_data, benign_df, attack_df, numeric_cols, scaler = load_and_preprocess_data(data_path)

C:\Users\jalna\AppData\Local\Temp\ipykernel_19848\216364003.py:5: DtypeWarning: Columns (0,1,3,6,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='latin1')


In [3]:
input_dim = benign_data.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim, ))

encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

history = autoencoder.fit(benign_data, benign_data,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_split=0.2,
                verbose=1)




Epoch 1/50

 676/7098 [=>............................] - ETA: 11s - loss: 0.0274

KeyboardInterrupt: 

In [4]:
from sklearn.metrics import roc_curve, auc

benign_pred = autoencoder.predict(benign_data)
attack_pred = autoencoder.predict(attack_data)

benign_mse = np.mean(np.power(benign_data - benign_pred, 2), axis=1)
attack_mse = np.mean(np.power(attack_data - attack_pred, 2), axis=1)

# Plotting reconstruction error distribution
plt.figure(figsize=(12, 6))
sns.histplot(benign_mse, bins=50, kde=True, color='blue', label='Benign')
sns.histplot(attack_mse, bins=50, kde=True, color='red', label='Attack')
plt.legend()
plt.title('Reconstruction Error Distribution')
plt.xlabel('Mean Squared Error')
plt.show()

# ROC Curve and AUC
true_labels = np.concatenate([np.zeros(len(benign_mse)), np.ones(len(attack_mse))])
pred_scores = np.concatenate([benign_mse, attack_mse])

fpr, tpr, thresholds = roc_curve(true_labels, pred_scores)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

KeyboardInterrupt: 

In [ ]:
## merge attack and benign dataframes
import ipaddress
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import scikeras
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score
import time

def convert_ip2int(ip):
    try:
        return int(ipaddress.ip_address(ip))
    except ValueError:
        return 0  # Return 0 for invalid IP addresses    

def preprocess_data(data):
    df = data.copy()
    df.columns = df.columns.str.strip().str.replace('[ /]', '_', regex=True)
    
    df['Source_IP'] = df['Source_IP'].apply(convert_ip2int)
    df['Destination_IP'] = df['Destination_IP'].apply(convert_ip2int)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce').astype(np.int64)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)

    y = np.where(df['Label'] == 'BENIGN', 0, 1)
    X = df.drop('Label', axis=1)
    X = X.select_dtypes(include=np.number)
    return X, y


In [6]:
X, y = preprocess_data(pd.concat([benign_df, attack_df], ignore_index=True))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"Train 데이터 크기: {X_train.shape}")
print(f"Test 데이터 크기: {X_test.shape}")
print("-" * 50)

Train 데이터 크기: (2262300, 83)
Test 데이터 크기: (565576, 83)
--------------------------------------------------


In [ ]:
## RF
print("Random Forest 모델 학습 시작...")
RF_starttime = time.time()
RF_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
RF_model.fit(X_train, y_train)
RF_endtime = time.time()
RF_time = RF_endtime - RF_starttime
print("Random Forest 모델 학습 완료.")

y_pred_RF = RF_model.predict(X_test)
accuracy_RF = accuracy_score(y_test, y_pred_RF)
recall_RF = recall_score(y_test, y_pred_RF)
f1_RF = f1_score(y_test, y_pred_RF)


In [ ]:
## LSTM
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_lstm = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_lstm = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

def create_lstm_model(lstm_units=100, dense_units=50, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential([
        LSTM(lstm_units, activation='relu',
        input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

print("LSTM 모델 학습 시작...")
LSTM_starttime = time.time()
lstm_model = KerasClassifier(build_fn=create_lstm_model)
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, verbose=1)
LSTM_endtime = time.time()
LSTM_time = LSTM_endtime - LSTM_starttime
print("LSTM 모델 학습 완료.")

y_pred_lstm = (lstm_model.predict(X_test_lstm) > 0.5).astype(int)
accuracy_lstm = accuracy_score(y_test, y_pred_lstm)
recall_lstm = recall_score(y_test, y_pred_lstm)
f1_lstm = f1_score(y_test, y_pred_lstm)



In [ ]:
## 모델 성능 비교
print("모델 성능 비교:")

print("[Random Forest]")
print(f"- 정확도: {accuracy_RF:.4f}")
print(f"- 재현율 (Recall): {recall_RF:.4f}")
print(f"- F1-Score: {f1_RF:.4f}")
print(f"- 학습 시간: {RF_time:.4f}초")
print("성능 리포트:")
print(classification_report(y_test, y_pred_RF, target_names=['BENIGN (0)', 'ATTACK (1)']))
print("-" * 50)

print("[LSTM]")
print(f"- 정확도: {accuracy_lstm:.4f}")
print(f"- 재현율 (Recall): {recall_lstm:.4f}")
print(f"- F1-Score: {f1_lstm:.4f}")
print(f"- 학습 시간: {LSTM_time:.4f}초")
print("성능 리포트:")
print(classification_report(y_test, y_pred_lstm, target_names=['BENIGN (0)', 'ATTACK (1)']))
print("-" * 50)